In [1]:
from pathlib import Path

import objaverse

In [8]:
folder = Path(f"{objaverse.BASE_PATH}/hf-objaverse-v1/glbs")
annotations = objaverse.load_annotations(
    [path.name.replace(".glb", "") for path in folder.glob("**/*.glb")]
)

 99%|█████████▊| 68/69 [00:25<00:00,  2.65it/s]


In [9]:
to_download = []
for value in annotations.values():
    n_face = value["faceCount"]
    if n_face <= 512 and n_face >= 16:
        # to_download.append(value["uid"])
        # (folder / f'{value["uid"]}.glb').unlink()
        to_download.append(value["uid"])

In [10]:
len(to_download)

21136

In [20]:
objects = objaverse.load_objects(to_download[20000:])

In [23]:
folder = Path("../data/objaverse/train")
# for uid, path in objects.items():
#     sln = folder / f"{uid}.glb"
#     sln.symlink_to(Path(path))

In [24]:
len(list(folder.glob("*.glb")))

20000